In [1]:
import torch
import dgl
import datetime

from dgl.data.utils import makedirs, save_info, load_info
from dgl.sampling import node2vec_random_walk
from sklearn.model_selection import train_test_split
from collections import Counter

import numpy as np
import pandas as pd
import seaborn as sb
import seaborn.objects as so

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)

# Append path
import sys
sys.path.append('../src')

import xgboost as xgb
from sklearn.metrics import roc_auc_score, average_precision_score, f1_score

from torch.optim import Adam
from torch.functional import F

import utils.utils_func
import experiment_multiround.supervised_multi
import models.benchmarks_supervised.simple
import models.benchmarks_supervised.spectral
import models.benchmarks_supervised.h2fd
import models.benchmarks_supervised.booster
import models.proposed_supervised.prototype

from experiment_multiround.supervised_multi import MultiroundExperiment
from models.benchmarks_supervised.simple import GCN
from models.benchmarks_supervised.spectral import BWGNN
from models.benchmarks_supervised.h2fd import H2FD
from models.benchmarks_supervised.booster import GIN_noparam
from models.proposed_supervised.prototype import ProtoFraud, SplitProtoFraud


In [2]:
torch.tensor([1, 2, 3, 4])[2:]

tensor([3, 4])

In [3]:
### DATASET STUFF
dataset, _ = dgl.load_graphs(f'../dataset/tfinance')
graph = dataset[0].long()

if len(graph.ndata['label'].shape) > 1:
    graph.ndata['label'] = graph.ndata['label'].argmax(1)
    graph.ndata['label'] = graph.ndata['label'].long().squeeze(-1)
graph.ndata['feature'] = graph.ndata['feature'].float()


In [4]:
in_feats = graph.ndata['feature'].shape[1]
num_classes = 2
h_feats = 10
num_layers = 2
num_proto = 3

model = SplitProtoFraud(in_feats, num_classes, h_feats, num_layers, num_proto=num_proto)

In [5]:
import time

start = time.time()
model(graph, graph.ndata['feature'])
print('Inference time', time.time() - start)

h torch.Size([42484443, 10])
lbl torch.Size([42484443])
Message passing time 0.687891960144043
Postprocessing time 0.004511356353759766
h torch.Size([42484443, 30])
lbl torch.Size([42484443])
Message passing time 6.1302220821380615
Postprocessing time 0.005001544952392578
torch.Size([1803, 90])
Full batch converged at iteration 23/100 with center shifts = tensor([0.]).
Full batch converged at iteration 17/100 with center shifts = tensor([0.]).
K-means time 2.182706594467163
Distance calc time 0.014237642288208008
Inference time 12.547052383422852
